# Lab 8: Experiments and Models

Welcome to lab 8! This week, we will be completing the hiring test that Enroll America used to screen their data analysts. Enroll America was a non-profit group that used data science to help sign people up for health insurance under the Affordable Care Act.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Bring in model data for Part 1
data_model = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/ea_test_model_analysis.csv")
data_model_dict = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/model_data_dict.csv")
# Bring in experiment data for Part 2
data_experiment = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/ea_test_experiment_analysis.csv")
data_experiment_dict = pd.read_csv("https://raw.githubusercontent.com/joshuakalla/data_science_campaigns/master/Colab/Lab8/experiment_data_dict.csv")

## Part 1: Model

The above file `data_model` contains a dataset that was in part used to build a predictive model identifying an individual’s likelihood of being uninsured (`uninsured_score`). A data dictionary is included in `data_model_dict`. The column `q1_healthcare` contains the results to a survey question asking if the respondent has health insurance or not.

Using the dataset, produce an evaluation of the model that will help someone understand how the model works. Some ideas to consider include basic summary information about the model, model validation, cross tabs of interesting demographic groups and useful visualizations. Be sure to calculate and report the accuracy, precision, and recall of the model.

First, take a look at the dictionary and the data.

In [ ]:
# Here is data_model
data_model.head()

In [ ]:
#pd.set_option('display.max_colwidth', -1) #https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin
# Here is the dictionary
data_model_dict

**Below, evaluate and interpret the model. You need a mix of both code and textual interpretation.** Be sure to calculate and report the accuracy, precision, and recall of the model.

You **should not** reverse engineer the model. Instead, think about precision/accuracy/recall as well as helpful visualizations, like from Lab 7. Apply and interpret those terms.

## Part 2: Experiment

The above file `data_experiment` contains a dataset with the results of a randomized controlled experiment conducted by a non-profit organization working on health insurance enrollment. A data dictionary is included in `data_experiment_dict`. 

For the experiment, the control group was suppressed from receiving any contact from the organization for a month. The treatment group was included in the regular program of the organization, which consists of two forms of contact. Most were called by a field organizer or volunteer and, if contacted, encouraged by phone to enroll in health insurance. Additionally, if an individual was subscribed to the email list they also received several emails a week encouraging them to enroll in health insurance. 

The most important outcome is whether an individual said they currently have health insurance when surveyed by phone 2 weeks after the end of the experiment. The results to the follow-up survey are in columns Q1 through Q15. Using the dataset, test whether the outreach efforts had any causal effect on insurance status in the treatment group. Is the insurance rate greater in the treatment group than in the control group? Is this difference statistically significant? Remember to use permutation tests, like in Lab 6.

First, take a look at the dictionary and the data.

In [ ]:
# Here is data_experiment
pd.set_option('display.max_columns', 43) #https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook
data_experiment.head()

In [ ]:
# Here is the dictionary
data_experiment_dict

**Below, analyze the experiment. You need a mix of both code and textual interpretation.**

## Part 3

Based on your analysis from Part 2, briefly propose a follow-up randomized controlled experiment. The proposed test should be designed to expand upon the knowledge derived from the first test.

**Enter your proposal here.**

# Congratulations!

You are done with the lab. Before you finish and submit, please fill out this brief evaluation:

- I spent around XXXX hours on this lab,.
- This lab was (too easy, too hard, just about the right difficulty).

**To turn in your lab, you will need to submit a PDF through Canvas.**